#### 제2유형 연습하기. iris 종 분류

In [153]:
# 라이브러리 불러오기

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score, mean_squared_error, r2_score

In [154]:
from sklearn.datasets import load_iris

iris = load_iris()

x = pd.DataFrame(iris.data, columns=['sepal_length', 'sepal_width', 'petal_length', 'petal_width'])
y = iris.target

# 이진 분류로 변경 (setosa = 0, 나머지 = 1)
# print(y)

y = np.where(y>0, 1, 0)
# print(y)

# 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=23)

x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
y_train = pd.DataFrame(y_train)

# 기존 넘파이 어레이에서 데이터 프레임으로 변경 후 컬럼명 지정
y_train.columns = ['species']

# 결측치 삽입
x_test['sepal_length'].iloc[0] = None
x_train['sepal_length'].iloc[0] = None

# 이상치 삽입
x_train['sepal_width'].iloc[0] = 150

#### 데이터 탐색(EDA)

In [155]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(120, 4)
(30, 4)
(120, 1)


In [156]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 117 to 51
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  119 non-null    float64
 1   sepal_width   120 non-null    float64
 2   petal_length  120 non-null    float64
 3   petal_width   120 non-null    float64
dtypes: float64(4)
memory usage: 4.7 KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 144 to 128
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  29 non-null     float64
 1   sepal_width   30 non-null     float64
 2   petal_length  30 non-null     float64
 3   petal_width   30 non-null     float64
dtypes: float64(4)
memory usage: 1.2 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   spe

In [157]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    1
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
species    0
dtype: int64


In [158]:
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

              count      mean        std  min  25%   50%   75%    max
sepal_length  119.0  5.801681   0.805488  4.3  5.1  5.70  6.35    7.9
sepal_width   120.0  4.275000  13.422352  2.0  2.8  3.00  3.40  150.0
petal_length  120.0  3.745000   1.758430  1.0  1.6  4.35  5.10    6.9
petal_width   120.0  1.186667   0.752467  0.1  0.3  1.30  1.80    2.5
              count     mean       std  min    25%   50%    75%  max
sepal_length   29.0  5.92069  0.867006  4.4  5.100  6.10  6.500  7.6
sepal_width    30.0  3.06000  0.321205  2.4  2.825  3.00  3.275  3.8
petal_length   30.0  3.81000  1.821964  1.3  1.500  4.40  5.175  6.6
petal_width    30.0  1.25000  0.811448  0.1  0.325  1.35  1.900  2.5
         count      mean       std  min  25%  50%  75%  max
species  120.0  0.666667  0.473381  0.0  0.0  1.0  1.0  1.0


In [159]:
# 결측치 제거
# 제거 전 drop 시킬지 대체할지 결정

x_train['sepal_length'] = x_train['sepal_length'].fillna(x['sepal_length'].median())
x_test['sepal_length'] = x_test['sepal_length'].fillna(x['sepal_length'].median())

# x_train.dropna(inplace=True)
# x_test.dropna(inplace=True)

print(x_train.isnull().sum())
print(x_test.isnull().sum())

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64
sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
dtype: int64


In [160]:
# 이상치 제거
# print(x_train.describe().T)
# print(x_test.describe().T)

# sepal_width에서 x_train의 max값 불량 확인
# Q1 = x_train['sepal_width'].quantile(.25)
# Q3 = x_train['sepal_width'].quantile(.75)
# IQR = abs(Q1-Q3)
# # print(Q1, Q3, IQR)

# upper = Q3+1.5*IQR
# lower = Q1-1.5*IQR

# cond1 = (x_train['sepal_width'] <= upper)
# cond2 = (x_train['sepal_width'] >= lower)


# print(x_train.describe().T)
# print(x_test.describe().T)

In [161]:
# 이상치 대체 (최대값으로 대체)

cond1 = (x_train['sepal_width'] <= 10)
max_sw = x_train[cond1]['sepal_width'].max()
# print(max_sw)

x_train['sepal_width'] = np.where(x_train['sepal_width'] >= 10, max_sw, x_train['sepal_width'])

print(x_train.describe().T)
print(x_test.describe().T)

              count      mean       std  min  25%   50%    75%  max
sepal_length  120.0  5.801667  0.802096  4.3  5.1  5.75  6.325  7.9
sepal_width   120.0  3.061667  0.472466  2.0  2.8  3.00  3.400  4.4
petal_length  120.0  3.745000  1.758430  1.0  1.6  4.35  5.100  6.9
petal_width   120.0  1.186667  0.752467  0.1  0.3  1.30  1.800  2.5
              count      mean       std  min    25%   50%    75%  max
sepal_length   30.0  5.916667  0.852211  4.4  5.125  5.95  6.500  7.6
sepal_width    30.0  3.060000  0.321205  2.4  2.825  3.00  3.275  3.8
petal_length   30.0  3.810000  1.821964  1.3  1.500  4.40  5.175  6.6
petal_width    30.0  1.250000  0.811448  0.1  0.325  1.35  1.900  2.5


In [162]:
# 모델링 적용

model = RandomForestClassifier()

model.fit(x_train, y_train)

C:\Users\user\AppData\Local\Temp\ipykernel_23032\2863692258.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestClassifier()

In [163]:
y_pred = model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred, average='macro')
roc_auc = roc_auc_score(y_test, y_pred)

print(accuracy)
print(f1_score)
print(roc_auc)

1.0
1.0
1.0


In [166]:
y_prob = model.predict_proba(x_test)
# print(y_prob)

In [169]:
result = pd.DataFrame({
  'result': y_pred,
  'prob_0': y_prob[:,0]
})

print(result[:5])

   result  prob_0
0       1    0.01
1       0    1.00
2       0    1.00
3       1    0.00
4       1    0.00


In [170]:
result.to_csv('test.csv',index=False)

In [171]:
print(pd.read_csv('test.csv').head())

   result  prob_0
0       1    0.01
1       0    1.00
2       0    1.00
3       1    0.00
4       1    0.00
